In [ ]:
!pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Success

In [ ]:
# !pip install shap

In [ ]:
import os
import zipfile
import urllib
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import ydf #Yggdrasil Decision Forest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
dummy_gform = 'https://github.com/iMnuelll/Bangkit_Capstone/raw/5571a4d9499be5716de713c9a13f9ea168440002/Data%20Dummy/dataset_akhir_final.csv'

data = pd.read_csv(dummy_gform, sep=',')
# data = pd.read_csv(data_dummy, sep='|')

In [ ]:
data.columns
print(data.dtypes)

gender                  int64
age                     int64
job                     int64
task                    int64
work_days               int64
difficulty              int64
average_work_hour       int64
average_rest_float    float64
average_rest            int64
average_rest_round      int64
mood_before_work        int64
mood_after_work         int64
deadline                int64
importance              int64
sleep_average           int64
urgency                 int64
total_gangguan          int64
productive_time         int64
work_hour_label         int64
dtype: object


In [ ]:
# Define the features and target label
features = [
            # 'gender',
            'age',
            # 'job',
            'task',
            # 'difficulty',
            'average_rest',
            # 'average_rest_round',
            'mood_before_work',
            # 'mood_after_work',
            'deadline',
            'importance',
            'sleep_average',
            'urgency',
            'total_gangguan',
            # 'work_hour_label',
            'average_work_hour',
            'work_days',
            ]

target = 'productive_time'

X = data[features]
y = data[target]

# Encode categorical variables
# X = pd.get_dummies(X)
# le = LabelEncoder()
# y = le.fit_transform(y)

print(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=2,
                                                    stratify=y,
                                                    )
# Standardize the features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

     age  task  average_rest  mood_before_work  deadline  importance  \
0     40     6             4                 1         3           3   
1     30     5             3                 2         4           4   
2     31     5             3                 0         4           3   
3     39     5             3                 1         3           1   
4     31     3             4                 1         4           4   
..   ...   ...           ...               ...       ...         ...   
322   20     6             1                 0         1           2   
323   17     6             1                 0         1           4   
324   21     6             1                 1         1           2   
325   20     6             1                 1         1           2   
326   19     6             1                 1         2           1   

     sleep_average  urgency  total_gangguan  average_work_hour  work_days  
0                8        3               3                

In [ ]:
train_data = X_train.copy()
train_data[target] = y_train

test_data = X_test.copy()
test_data[target] = y_test

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label=target)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label=target)

In [ ]:
results = []
best_accuracy = 0
best_config = {}

for max_depth in [None, 10, 20, 30]:
    for min_examples in [1, 2, 5, 10]:
        for num_trees in [50, 100, 200]:
            for compute_oob in [True, False]:
                model_gbt = tfdf.keras.RandomForestModel(
                    max_depth=max_depth,
                    min_examples=min_examples,
                    num_trees=num_trees,
                    compute_oob_variable_importances=compute_oob,
                    verbose=0
                )
                model_gbt.compile(metrics=['accuracy'])
                model_gbt.fit(x=train_ds)
                evaluation = model_gbt.evaluate(test_ds, return_dict=True)
                accuracy = evaluation['accuracy']
                results.append((accuracy, max_depth, min_examples, num_trees, compute_oob))
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_config = {
                        'max_depth': max_depth,
                        'min_examples': min_examples,
                        'num_trees': num_trees,
                        'compute_oob_variable_importances': compute_oob
                    }
                # Print the parameters and the accuracy for each iteration
                print(f"Max Depth: {max_depth}, Min Examples: {min_examples}, Num Trees: {num_trees}, Compute OOB: {compute_oob}, Accuracy: {accuracy}")

# Print the best configuration and its accuracy
print(f"Best Configuration: {best_config}")
print(f"Best Accuracy: {best_accuracy}")
# print(f"Best Accuracy: {best_accuracy:.4f}")

1/1 [==============================] - 5s 5s/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 1, Num Trees: 50, Compute OOB: True, Accuracy: 0.8636363744735718
1/1 [==============================] - 0s 250ms/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 1, Num Trees: 50, Compute OOB: False, Accuracy: 0.8636363744735718
1/1 [==============================] - 0s 248ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 1, Num Trees: 100, Compute OOB: True, Accuracy: 0.8333333134651184
1/1 [==============================] - 0s 266ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 1, Num Trees: 100, Compute OOB: False, Accuracy: 0.8333333134651184


1/1 [==============================] - 0s 378ms/step - loss: 0.0000e+00 - accuracy: 0.8485
Max Depth: None, Min Examples: 1, Num Trees: 200, Compute OOB: True, Accuracy: 0.8484848737716675


1/1 [==============================] - 1s 995ms/step - loss: 0.0000e+00 - accuracy: 0.8485
Max Depth: None, Min Examples: 1, Num Trees: 200, Compute OOB: False, Accuracy: 0.8484848737716675
1/1 [==============================] - 0s 259ms/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 2, Num Trees: 50, Compute OOB: True, Accuracy: 0.8636363744735718
1/1 [==============================] - 1s 595ms/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 2, Num Trees: 50, Compute OOB: False, Accuracy: 0.8636363744735718
1/1 [==============================] - 0s 263ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 2, Num Trees: 100, Compute OOB: True, Accuracy: 0.8333333134651184
1/1 [==============================] - 0s 253ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 2, Num Trees: 100, Compute OOB: False, Accuracy: 0.8333333134651184
1/1 [==============================] - 0s 249ms/step -

In [ ]:
# model_gbt = tfdf.keras.GradientBoostedTreesModel()
model_gbt = tfdf.keras.RandomForestModel(
    max_depth=10,
    min_examples=5,
    num_trees=100,
    verbose=1,
    compute_oob_variable_importances=True
    )

model_gbt.fit(x=train_ds)

Use /tmp/tmp3ptyhyir as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.488268. Found 261 examples.
Training model...
Model trained in 0:00:00.156687
Compiling model...
Model compiled.


In [ ]:
model_gbt.compile(metrics=['accuracy'])

evaluation  = model_gbt.evaluate(train_ds, return_dict = True)

for name, value in evaluation.items() :
  print(f"train {name}: {value:.4f}")

evaluation  = model_gbt.evaluate(test_ds, return_dict = True)

for name, value in evaluation.items() :
  print(f"test {name}: {value:.4f}")

1/1 [==============================] - 0s 306ms/step - loss: 0.0000e+00 - accuracy: 0.9464
train loss: 0.0000
train accuracy: 0.9464
1/1 [==============================] - 0s 154ms/step - loss: 0.0000e+00 - accuracy: 0.8788
test loss: 0.0000
test accuracy: 0.8788


In [ ]:
inspector = model_gbt.make_inspector()
feature_importances = inspector.variable_importances()

In [ ]:
for importance_type, importance_list in feature_importances.items():
    print(f"\nImportance type: {importance_type}")
    for feature_importance in importance_list:
        print(f"  Feature: {feature_importance[0]}, Importance: {feature_importance[1]}")


Importance type: MEAN_DECREASE_IN_AUC_3_VS_OTHERS
  Feature: "age" (1; #1), Importance: 0.18531468531468565
  Feature: "sleep_average" (1; #7), Importance: 0.03939393939393887
  Feature: "deadline" (1; #4), Importance: 0.02777777777777768
  Feature: "average_work_hour" (1; #3), Importance: 0.019153069153069135
  Feature: "urgency" (1; #10), Importance: 0.008974358974358831
  Feature: "mood_before_work" (1; #6), Importance: 0.004079254079253802
  Feature: "total_gangguan" (1; #9), Importance: 0.0008158508158505606
  Feature: "importance" (1; #5), Importance: -0.0015151515151513584
  Feature: "work_days" (1; #11), Importance: -0.0021756021756026422
  Feature: "average_rest" (1; #2), Importance: -0.005788655788656283
  Feature: "task" (1; #8), Importance: -0.008935508935508873

Importance type: MEAN_DECREASE_IN_ACCURACY
  Feature: "age" (1; #1), Importance: 0.10344827175140381
  Feature: "deadline" (1; #4), Importance: 0.10344827175140381
  Feature: "sleep_average" (1; #7), Importance: 0

**Penjelasan Importance Keys :**

https://medium.com/@shahcheril311/training-tree-based-models-with-tf-decision-forests-5eb98860c155

In [ ]:
print(model_gbt.summary())

Model: "random_forest_model_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	age
	average_rest
	average_work_hour
	deadline
	importance
	mood_before_work
	sleep_average
	task
	total_gangguan
	urgency
	work_days

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.     "sleep_average"  0.345398 ################
    2.               "age"  0.307542 ############
    3.          "deadline"  0.298218 ############
    4. "average_work_hour"  0.207036 ####
    5.         "work_days"  0.174109 #
    6.    "total_gangguan"  0.161615 
    7.              "task"  0.160582 
    8.           "urgency"  0.159757 
    9.      "average_rest"  0.158300 
   10.  "mood_be

In [ ]:
# Contoh data untuk prediksi
predict_data = {
    'age': [29],
    'task': [5],
    'average_rest': [30],
    'mood_before_work': [3],
    'deadline': [2],
    'importance': [3],
    'sleep_average': [7],
    'urgency': [1],
    'total_gangguan': [0],
    'average_work_hour': [8],
    'work_days': [5]
}

# konversi data ke DataFrame pandas
predict_df = pd.DataFrame(predict_data)
print(predict_df.info())
print(predict_df.head())
print("end")

# konversi DataFrame ke TensorFlow Dataset
predict_ds = tfdf.keras.pd_dataframe_to_tf_dataset(predict_df)

for example in predict_ds.take(1):  # Mengambil satu batch
    print(example)
print("end")

# Melakukan prediksi
predictions = model_gbt.predict(predict_ds)

# print("Predictions:", predictions)
# print(predictions.dtype)

print("Predictions:", predictions)
print("Type of predictions:", type(predictions))
print("Shape of predictions:", predictions.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                1 non-null      int64
 1   task               1 non-null      int64
 2   average_rest       1 non-null      int64
 3   mood_before_work   1 non-null      int64
 4   deadline           1 non-null      int64
 5   importance         1 non-null      int64
 6   sleep_average      1 non-null      int64
 7   urgency            1 non-null      int64
 8   total_gangguan     1 non-null      int64
 9   average_work_hour  1 non-null      int64
 10  work_days          1 non-null      int64
dtypes: int64(11)
memory usage: 216.0 bytes
None
   age  task  average_rest  mood_before_work  deadline  importance  \
0   29     5            30                 3         2           3   

   sleep_average  urgency  total_gangguan  average_work_hour  work_days  
0              7        1            

In [ ]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
model_gbt.save('chronotype_model')

In [ ]:
!pip install tensorflowjs  # Memastikan tensorflowjs terinstal
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model ./chronotype_model ./chronotype_model_tfjs

2024-06-12 04:58:15.978529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO 24-06-12 04:58:20.4651 UTC kernel.cc:1233] Loading model from path ./chronotype_model/assets/ with prefix c3f1cc047ca548ff
[INFO 24-06-12 04:58:20.4895 UTC decision_forest.cc:734] Model loaded with 100 root(s), 6274 node(s), and 11 input feature(s).
[INFO 24-06-12 04:58:20.4896 UTC abstract_model.cc:1362] Engine "RandomForestGeneric" built
[INFO 24-06-12 04:58:20.4897 UTC kernel.cc:1061] Use fast generic engine
weight StatefulPartitionedCall/random_forest_model_97/StatefulPartitionedCall/RaggedConstant/Const with shape (1,) and dtype int64 was auto converted to the type int32
weight StatefulPartitionedCall/random_forest_model_97/StatefulPartitionedCall/RaggedConstant/Const_1 with shape (1,) and dtype int64 was auto converted to the type int32


In [ ]:
import shutil

# Path ke folder model TensorFlow.js
folder_path = './chronotype_model_tfjs'
# Nama file zip output
zip_path = './chronotype_model_tfjs.zip'

# Membuat zip file dari folder model
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)


'/content/chronotype_model_tfjs.zip'

In [ ]:
from google.colab import files

# Download file zip
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

# Path ke folder SavedModel
folder_path = './chronotype_model'
# Nama file zip output
zip_path = './chronotype_model_savedmodel.zip'

# Membuat zip file dari folder SavedModel
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)


'/content/chronotype_model_savedmodel.zip'

In [ ]:
from google.colab import files
# Download file zip
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load SavedModel

In [ ]:
# Download SavedModel dari GitHub, kalo gak mau repot jalanin kode di atas buat
# dapetin SavedModel nya

url = 'https://raw.githubusercontent.com/iMnuelll/Bangkit_Capstone/main/Deployment_Utilities/chronotype_model_savedmodel.zip'
urllib.request.urlretrieve(url, 'chronotype_model_savedmodel.zip')

destination_folder = 'chronotype_model_savedmodel'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

with zipfile.ZipFile(destination_folder + '.zip', 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

NameError: name 'urllib' is not defined

In [ ]:
# Path ke model yang disimpan
model_path = './chronotype_model_savedmodel'

# Mendapatkan fungsi inferensi dari model yang dimuat
model = tf.saved_model.load(model_path)

predict_data = {
    'age': [29],
    'task': [5],
    'average_rest': [30],
    'mood_before_work': [3],
    'deadline': [2],
    'importance': [3],
    'sleep_average': [7],
    'urgency': [1],
    'total_gangguan': [0],
    'average_work_hour': [8],
    'work_days': [5]
}

# Get the prediction function from the loaded model
# Assuming your model has a single signature named 'serving_default'
infer = model.signatures['serving_default']

# Prepare input data as a dictionary of TensorFlow tensors
input_tensors = {
    key: tf.constant(value, dtype=tf.int64)  # Adjust dtype if necessary
    for key, value in predict_data.items()
}

print("data input to the model")
print(input_tensors, "\n")

# Perform prediction, passing inputs as keyword arguments
output = infer(**input_tensors)

# Print the output
print("data output from the model")
print(output, "\n")

# Akses tensor dari kamus output menggunakan kunci yang sesuai
tensor = output['output_1']

print("access the tensor of the output dictionary from the model")
print(tensor, "\n")

# Konversi tensor ke array NumPy
array = tensor.numpy()

# Hasil array
print("array or prediction from the model")
print(array)

data input to the model
{'age': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([29])>, 'task': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([5])>, 'average_rest': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([30])>, 'mood_before_work': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([3])>, 'deadline': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([2])>, 'importance': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([3])>, 'sleep_average': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([7])>, 'urgency': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>, 'total_gangguan': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, 'average_work_hour': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([8])>, 'work_days': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([5])>} 

data output from the model
{'output_1': <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.17      , 0.55999976, 0.21000002, 0.05999999]], dtype=float32)>} 

access the tensor of the output di

In [ ]:
#Pake library yggdrasil decision forest = YDF
# katanya YDF lebih bagus dari TFDF
# gimana tuh kira-kira? :v

model_path = './chronotype_model_savedmodel'

model_from_tfdf = ydf.from_tensorflow_decision_forests(model_path)

predict_data = {
    'age': [29],
    'task': [5],
    'average_rest': [30],
    'mood_before_work': [3],
    'deadline': [2],
    'importance': [3],
    'sleep_average': [7],
    'urgency': [1],
    'total_gangguan': [0],
    'average_work_hour': [8],
    'work_days': [5]
}

print(model_from_tfdf.predict(predict_data))

The model was created by Tensorflow Decision Forests and it contains integerized columns. A dictionary has automatically been added to the model's dataspec.
[[0.17       0.55999976 0.21000002 0.05999999]]


In [ ]:
model_from_tfdf.save('chronotype_model_ydf')

import shutil

# Path ke folder SavedModel
folder_path = './chronotype_model_ydf'
# Nama file zip output
zip_path = './chronotype_model_ydf.zip'

# Membuat zip file dari folder SavedModel
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

'/content/chronotype_model_ydf.zip'

In [ ]:
!pip show tensorflow_decision_forests

ERROR: Operation cancelled by user
